In [9]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import json
import pandas as pd
import numpy as np # for NaN values
import os


In [2]:
# Set environment variables
PLACES_KEY =  os.getenv('PLACES_API_KEY')
# Set service
service = build('places', 'v1', developerKey=PLACES_KEY)

In [3]:
def places_connection(api_key):
    try:
        # Connect to Google Places API
        service = build('places', 'v1', developerKey=api_key)        
               
        return service
    
    except HttpError as error:
        print(f'An error occurred: {error}')
        return None

In [4]:
service = places_connection(PLACES_KEY)

In [5]:
# Load cities
cities = pd.read_csv('tests\cities.csv')
cities[:3]

,city
0,Altamonte-Springs
1,Apopka
2,Bartow


## Where to eat

In [6]:
where_to_eat = ['bar_and_grill', 'restaurant', 'cafe']

In [10]:
# Loop through each city
def get_places(cities, where_to_eat):
    stay_list = []  # List to store places to stay

    for city in cities:

        for place in where_to_eat:

            try:
                # Prepare the initial search query
                query = f'{place} in {city}, Florida, USA'
                request_body = {
                    'textQuery': query,
                    'regionCode': 'US',
                    'languageCode':'en',
                    'pageSize': 20
                }
                
                # Make the request to the   
                response = service.places().searchText(body=request_body, fields='*').execute()
                
                # Process the current page of results
                if 'places' in response:
                    for place in response['places']:
                        store_data = {
                                    'City': city,
                                    'Where to eat': place.get('displayName', {}).get('text', 'N/A'),
                                    'Address': place.get('formattedAddress', 'N/A'),
                                    'Latitude': place.get('location', {}).get('latitude', np.nan),
                                    'Longitude': place.get('location', {}).get('longitude', np.nan),
                                    'Rating': place.get('rating', np.nan),
                                    'User Rating Count': place.get('userRatingCount', 0),
                                    'Comments': [
                                        review.get('text', 'N/A') for review in place.get('reviews', [])[:3]
                                    ]  # Extraer hasta los 3 mejores comentarios
                                }

                        stay_list.append(store_data)                    
                        
            except Exception as e:
                    print(f"Error processing {city}: {e}")                
                    continue

            # Convert to DataFrames
        df = pd.DataFrame(stay_list)

    return df

In [11]:
df = get_places(cities['city'], where_to_eat)

In [14]:
df.sample(5)

,City,Where to eat,Address,Latitude,Longitude,Rating,User Rating Count,Comments
475,Lake-Buena-Vista,Everglazed Donuts & Cold Brew,"1504 E Buena Vista Dr, Lake Buena Vista, FL 32830",28.369860,-81.521216,4.4,1224,"[{'text': 'Morning of December 8th, 2024 Abso..."
780,Ocala,Ivy on the Square,"53 S Magnolia Ave, Ocala, FL 34471",29.185633,-82.136855,4.4,2015,"[{'text': 'We enjoyed our food here, and every..."
161,Bartow,Tropical Smoothie Cafe,"1490 N Broadway Ave, Bartow, FL 33830",27.908444,-81.842833,4.3,423,[{'text': 'The absolute best customer service ...
1148,Tavares,Gator Bay Bar and Grill,"10320 County Rd 44, Leesburg, FL 34788",28.872788,-81.785014,4.4,963,[{'text': 'If you enjoy a backwoods restaurant...
1305,Wildwood,City Fire American Oven & Bar,"2716 Brownwood Blvd, The Villages, FL 32163",28.844447,-82.021521,4.3,1328,"[{'text': 'Cool spot, nice interior vibe. Als..."


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1531 entries, 0 to 1530
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               1531 non-null   object 
 1   Where to eat       1531 non-null   object 
 2   Address            1531 non-null   object 
 3   Latitude           1531 non-null   float64
 4   Longitude          1531 non-null   float64
 5   Rating             1529 non-null   float64
 6   User Rating Count  1531 non-null   int64  
 7   Comments           1531 non-null   object 
dtypes: float64(3), int64(1), object(4)
memory usage: 95.8+ KB


In [16]:
# Filtering the data    
df_f = df[(df['Rating'] > 3.9) & (df['User Rating Count'] > 100)]

In [17]:
df_f.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1305 entries, 0 to 1530
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               1305 non-null   object 
 1   Where to eat       1305 non-null   object 
 2   Address            1305 non-null   object 
 3   Latitude           1305 non-null   float64
 4   Longitude          1305 non-null   float64
 5   Rating             1305 non-null   float64
 6   User Rating Count  1305 non-null   int64  
 7   Comments           1305 non-null   object 
dtypes: float64(3), int64(1), object(4)
memory usage: 91.8+ KB


In [18]:
df_f = df_f.drop_duplicates(subset='Latitude')

In [19]:
df_f.info()

<class 'pandas.core.frame.DataFrame'>
Index: 907 entries, 0 to 1530
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               907 non-null    object 
 1   Where to eat       907 non-null    object 
 2   Address            907 non-null    object 
 3   Latitude           907 non-null    float64
 4   Longitude          907 non-null    float64
 5   Rating             907 non-null    float64
 6   User Rating Count  907 non-null    int64  
 7   Comments           907 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 63.8+ KB


## NODE Where to eat

In [21]:
df_f.to_csv(r'neo4j\nodes\node_where_to_eat.csv', index=False, encoding='utf-8')

In [22]:
df_r = df_f[['Where to eat', 'City']].copy()

In [23]:
df_r.to_csv(r'neo4j\relationships\relationship_where_to_eat.csv', index=False, encoding='utf-8')    